# Population Based Training with RAY TUNE

Hyperparameter tuning is a key step in model selection. Hyperparameters are like settings, if you do not handle them appropriately, it can have a bad impact on the results of the model. Tuning can be done manually or automatically. In today's world, because of computational capabilities, a high number of hyperparameters, a big variety of algorithms, and helper libraries like the Ray, the preferred way is automatically tuning hyperparameters.

In this article, we'll talk about Population Based Training, explore Ray Tune, and see an example of hyperparameter tunning.

*What PBT means*

As we have already mentioned, the good performance of the model is related to the correct selection of hyperparameters. Population Based Training is one of the charming ways of hyperparameters choosing. It consists of two parts: random search and clever choosing. In the random search step, the algorithm chooses several combinations of hyperparameters randomly. There is a high chance that most of the combinations will have low-performance scores and a small portion of combinations on the contrary will have better / good performance scores. Here comes clever choosing. The second step is in a cycle until we achieve the desired result or we do not exhaust the number of iterations. The clever choosing step contains two main methods: *exploit* and *explore*. *Exploit* - replace the combination of the hyperparameters with more promising ones, based on the performance metric. *Explore* - randomly perturb the hyperparameters (in most cases it is multiplied by some factor) to add noise.   

Population Based Training allows doing two meaningful things together: parallelize training of hyperparameters combinations, study from the rest of the population and get promising results promptly.

*Talk about Tune*

Ray Tune is a Ray-based python library for hyperparameter tuning with the latest algorithms such as PBT. We will work on Ray version 2.1.0. Changes can be seen in release notes - https://github.com/ray-project/ray/releases. We will also mention important changes in the way. 

Before moving on to practical examples let's go over some basic concepts. *Tranable* - is an objective that helps algorithms to evaluate configurations. It can have Class API or Function API, but according to the ray tune documentation, Function API is recommended. *Search Spaces* - values ranges for hyperparameters. *Trials* - Tuner will generate several configurations and run processes on them, so the process runed on a configuration is called Trial. *Search Algorithm* - suggests hyperparameter configurations, by default Tune uses random search as the search algorithm. *Scheduler* - Based on reported results during training process, schedulers decide whether stop or continue. Next meaningful concept is *checkpointing*. Checkpointing means saving intermediate results, necessary to resume and then continue training.    

In most cases Search Algorithm and Scheduler can be used together in tuning process, but there is exception. One of the cases, when they are not used together is Population Based Training. In Ray Tune docs PBT is in schedulers part, but it is both at the same time. It's a scheduler because it stops trials based on the results and is a searcher as it has the logic to create a new configuration. Schedulers compatibility can be seen here - https://docs.ray.io/en/latest/tune/key-concepts.html#schedulers

We use well known The Boston Housing Dataset (https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). We can import this dataset from sklearn.

One meaningful change in Ray Tune was the execution API. tune.run() has changed into Tuner().fit. Before the update, we were passing the parameters separately, but in the new version, config classes were introduced which simplifies a lot of things. First of all, grouped related parameters together which makes execution code easier to read and understand. And second, when you use Ray Tune in a project, some configurations are the same for some algorithms, so you can make one common config class object and move around algorithms, which makes life easier. 

In [ ]:
# Install a pip package
import sys
!{sys.executable} -m pip install ray==2.1.0

In [1]:
# imports
import json
import os

from joblib import dump, load
from lightgbm import LGBMRegressor
from ray import tune
from ray.air.config import CheckpointConfig
from ray.air.config import RunConfig
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Let's start with trainable. As we already mentioned, there are two trainable APIs: function-based and class-based. We will write trainable with Class API.

In [17]:
class TrainableForPBT(tune.Trainable):
    def setup(self, config):
        pass
    
    def step(self):
        pass

tune.Trainable is base class for class-based trainables. We need to override at least two methods: <font color=blue>setup</font> and <font color=blue>step</font>. <font color=blue>Setup</font> is invoked when training starts and <font color=blue>step</font> on every iteration. Unlike <font color=blue>setup</font> function, <font color=blue>step</font> can be invoked several times. 

In the <font color=blue>setup</font> we need to have x_train and y_train to estimate the efficiency of the trial model in future steps. Of course, <font color=blue>setup</font> is the parent class's (tune.Trainable) function, but it gives us the possibility to add additional arguments. Also, we need to initialize the lgbm regressor/model in the <font color=blue>setup</font>. We are going to retrain our model on every iteration, but on the first one we want to just fit the model, hence need to count on which iteration are we in. Nothing more at this point.

In [1]:
def setup(self, config, x_train, y_train):
    self.current_config = config
    self.x_train = x_train
    self.y_train = y_train
    # need to add about model
    self.model = LGBMRegressor(**self.current_config)
    # count on which iteration are we in
    self.current_iteration = 0
    self.current_step_score = None

What do we do in <font color=blue>step</font>? We should estimate current configurations efficiency and return score. We will use cross-validation with r2 and return this score. Therefore, PBT will know scores associated with all configurations after each iterations and it will make decisions about perturbation based on these scores. Also we should refit model one more time if it is not first iteration.

In [7]:
def step(self):
    self.current_iteration += 1
    if self.current_iteration == 1:
        self.model.fit(self.x_train, self.y_train)
    else:
        self.model.fit(self.x_train, self.y_train, init_model=self.model)

    self.current_step_score = cross_val_score(estimator=self.model, X=self.x_train, y=self.y_train,
                                              scoring='r2', cv=5).mean()
    results_dict = {"r2_score": self.current_step_score}
    return results_dict

After two main function overrideing PBT needs more functions to override. For the exploition process we need to save and read checkpoint.

Start with <font color=blue>save_checkpoint</font>. We will use joblib library for saving and restoring model. What do we need to save? First of all - model, since we always need previous iteration model (init_model) for next iteration, we can also save current iteration number and current step score.

In [5]:
def save_checkpoint(self, tmp_checkpoint_dir):
    path = os.path.join(tmp_checkpoint_dir, "checkpoint")
    with open(path, "w") as f:
        f.write(json.dumps(
            {"current_score": self.current_step_score, "current_step": self.current_iteration}))

    path_for_model = os.path.join(tmp_checkpoint_dir, 'model.joblib')
    dump(self.model, path_for_model)

    return tmp_checkpoint_dir

We should restore same things from <font color=blue>load_checkpoint</font>.

In [6]:
def load_checkpoint(self, tmp_checkpoint_dir):
    with open(os.path.join(tmp_checkpoint_dir, "checkpoint")) as f:
        state = json.loads(f.read())
        self.current_step_score = state["current_score"]
        self.current_iteration = state["current_step"]

    path_for_model = os.path.join(tmp_checkpoint_dir, 'model.joblib')
    self.model = load(path_for_model)

Above trainable class can be considered as completed. But we can improve the training time with the reuse_actor feature. 

In the training process, we have as many Trainables as configuration samples. Each Trainable needs several seconds to start. With the reuse_actor feature, we can reuse already started Trainable for new multiple configurations/hyperparameters. So we will need less Trainable and the time spent on the start-up will be less as well.

Let's implement <font color=blue>reset_config</font>, which delivers new hyperparameters. In <font color=blue>reset_config</font> every variable need to be adjusted to new hyperparameters, it's like new <font color=blue>setup</font>. There is one tricky question, every time different configurations swap the same Trainable, do they start the process from scratch, due to the fact that in reset_config we write it like the start? Actually, no, because after reset_config, the Trainable calls <font color=blue>load checkpoint</font> if one exists, hence, training will continue from the last stop/checkpoint.

In [2]:
def reset_config(self, new_config):
    self.current_config = new_config
    self.current_iteration = 0
    self.current_step_score = None
    self.model = LGBMRegressor(**self.current_config)
    return True

We have completed the implementation of Trainable. The finished class will look like this

In [3]:
class TrainableForPBT(tune.Trainable):
    def setup(self, config, x_train, y_train):
        self.current_config = config
        self.x_train = x_train
        self.y_train = y_train
        # need to add about model
        self.model = LGBMRegressor(**self.current_config)
        # count on which iteration are we in
        self.current_iteration = 0
        self.current_step_score = None

    def step(self):
        self.current_iteration += 1
        if self.current_iteration == 1:
            self.model.fit(self.x_train, self.y_train)
        else:
            self.model.fit(self.x_train, self.y_train, init_model=self.model)

        self.current_step_score = cross_val_score(estimator=self.model, X=self.x_train, y=self.y_train,
                                                  scoring='r2', cv=5).mean()
        results_dict = {"r2_score": self.current_step_score}
        return results_dict

    def save_checkpoint(self, tmp_checkpoint_dir):
        path = os.path.join(tmp_checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps(
                {"current_score": self.current_step_score, "current_step": self.current_iteration}))

        path_for_model = os.path.join(tmp_checkpoint_dir, 'model.joblib')
        dump(self.model, path_for_model)

        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        with open(os.path.join(tmp_checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            self.current_step_score = state["current_score"]
            self.current_iteration = state["current_step"]

        path_for_model = os.path.join(tmp_checkpoint_dir, 'model.joblib')
        self.model = load(path_for_model)

    def reset_config(self, new_config):
        self.current_config = new_config
        self.current_iteration = 0
        self.current_step_score = None
        self.model = LGBMRegressor(**self.current_config)
        return True

Now we can create some configurations and run Tune experiment. Tuner has 4 parameter: trainable, param_space, tune_config and run_config. Trainable is already implemented. Let's define param_space.

Param_space is same as already mentioned search space. First, we need to define a list of parameters that we are going to tune. To simplify, choose 3 parameters: learning_rate, num_leaves, max_depth.

Tune has own Search Space API, so we should use them when we define search spaces. The name of search spaces are intuitive, so let's see the result without further ado.

In [4]:
param_space = {
    "params": {
        "learning_rate": tune.loguniform(1e-5, 1e-1), #between 0.00001 and 0.1
        "num_leaves":  tune.randint(5, 100), #between 5 and 100(exclusive)
        "max_depth": tune.randint(1, 9), #between 1 and 9(exclusive)
    },
}

Next thing to define is tune_config. But before we implement that, we need to create scheduler - Population Based Training object.

The first parameter of Population Base Training schedule is time_attr. It is the training result attribute for comparison, which should be something that increases monotonically. We choose trainig_iteration as a time attribute, so when we mention time_attr anywhere, that means training_iteration. perturbation_interval - how often should the perturbation occur. If we do perturbation often, then we need to save the checkpoints often as well. Hereby, let's choose perturbation_interval to be 4. burn_in_period - perturbation will not happen before this number of intervals (iteration) has passed. It will not be true if we clone the state of the top performers to poorly performing models from the very beginning, as performance scores are unstable at early stages. So give 10 iterations of burn_in_period trials and then start perturbation. hyperparam_mutations is a dict of hyperparameters and their search spaces, which can be perturbated. We want to perturbate all hyperparameters from the param_space dict, so hyperparam_mutations will be the same as param_space["params"]. We will not pass mode and metric arguments in PopulationBasedTraining, as we define them in TuneConfig.

In [5]:
pbt = PopulationBasedTraining(
            time_attr="training_iteration",
            perturbation_interval=4,
            burn_in_period=10,
            hyperparam_mutations=param_space["params"],
        )

In TuneConfig we need to pass metric, which is the name of reported score from trainable - "r2_score" in our case. Also mode, which has two value min or max, depends on objective minimizing or maximizing the metric. As already mentioned, we don't have a search algorithm and our scheduler algorithm is pbt (PopulationBasedTraining). reuse_actors should be true as well. num_samples - number of samples of hyperparameters from search space should tune.

In [6]:
tune_config = TuneConfig(metric="r2_score", mode="max", search_alg=None, scheduler=pbt, num_samples=15, reuse_actors=True)

Next to define is RunConfig, which contains CheckpointConfig inside, so first create CheckpointConfig and then RunConfig. In CheckpointConfig, checkpoint_score_attribute and checkpoint_score_order are the same as metric and mode in TuneConfig. Choose checkpoint_frequency same as perturbation_interval. Also, save the last checkpoint at the end of training with checkpoint_at_end=True.

In [7]:
checkpoint_config = CheckpointConfig(checkpoint_score_attribute="r2_score", 
                                     checkpoint_score_order="max", 
                                     checkpoint_frequency=4, 
                                     checkpoint_at_end=True)

In Run Config we can pass the name of experment and local_dir, which is directory where training results are saved. It will be useful if we would like to restore/continue the experiment in the future. We should add easy criteria for stopping - stop after 30 iterations.

In [8]:
run_config = RunConfig(name="pbt_experiment", 
                       local_dir='/Users/admin/Desktop/Dressler/Publications',
                       stop={"training_iteration": 30},
                       checkpoint_config=checkpoint_config)

It's time to create Tuner. Because we have trainable with extra arguments, we need to use and pass tune.with_parameters inside Tuner.

In [9]:
X, y = load_boston(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
trainable_with_parameters = tune.with_parameters(TrainableForPBT, x_train=x_train, y_train=y_train)

In [11]:
tuner = Tuner(trainable_with_parameters, param_space=param_space["params"], tune_config=tune_config, run_config=run_config)
analysis = tuner.fit()

2022-11-21 23:09:27,275	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Trial name,status,loc,learning_rate,max_depth,num_leaves,iter,total time (s),r2_score
TrainableForPBT_05d17_00000,TERMINATED,127.0.0.1:67960,0.0374349,3,52,30,12.3024,0.809821
TrainableForPBT_05d17_00001,TERMINATED,127.0.0.1:67961,0.000636968,6,86,30,14.6924,0.820332
TrainableForPBT_05d17_00002,TERMINATED,127.0.0.1:67959,0.00137336,5,31,30,16.7138,0.176872
TrainableForPBT_05d17_00003,TERMINATED,127.0.0.1:67950,0.0311957,4,44,30,12.645,0.809821
TrainableForPBT_05d17_00004,TERMINATED,127.0.0.1:67960,0.0296524,6,86,30,15.5177,0.820332
TrainableForPBT_05d17_00005,TERMINATED,127.0.0.1:67961,0.00164803,6,97,30,17.2069,0.176872
TrainableForPBT_05d17_00006,TERMINATED,127.0.0.1:67950,0.0374349,3,35,30,12.93,0.809821
TrainableForPBT_05d17_00007,TERMINATED,127.0.0.1:67959,1.55769e-05,4,52,30,12.7778,0.809821
TrainableForPBT_05d17_00008,TERMINATED,127.0.0.1:67960,0.0417284,5,72,30,15.5693,0.820332
TrainableForPBT_05d17_00009,TERMINATED,127.0.0.1:67950,0.0333827,4,57,30,15.5319,0.820332


2022-11-21 23:09:31,201	WARNING trial_runner.py:1575 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0


Result for TrainableForPBT_05d17_00000:
  date: 2022-11-21_23-09-36
  done: false
  experiment_id: c01173ebf45c4e309b7e8cf37b701cc6
  hostname: localhost
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 67950
  r2_score: 0.0005985651130453462
  time_since_restore: 0.3830301761627197
  time_this_iter_s: 0.3830301761627197
  time_total_s: 0.3830301761627197
  timestamp: 1669057776
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 05d17_00000
  warmup_time: 0.008919954299926758
  
Result for TrainableForPBT_05d17_00000:
  date: 2022-11-21_23-09-37
  done: false
  experiment_id: c01173ebf45c4e309b7e8cf37b701cc6
  hostname: localhost
  iterations_since_restore: 3
  node_ip: 127.0.0.1
  pid: 67950
  r2_score: 0.0005985651130453462
  time_since_restore: 1.074735403060913
  time_this_iter_s: 0.3020331859588623
  time_total_s: 1.074735403060913
  timestamp: 1669057777
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: 05d17_00000
  warmup_time: 0.0089199542

2022-11-21 23:09:44,738	INFO pbt.py:552 -- [pbt]: no checkpoint for trial. Skip exploit for Trial TrainableForPBT_05d17_00000


...

(TrainableForPBT pid=67961) 2022-11-21 23:10:54,555	INFO trainable.py:668 -- Restored on 127.0.0.1 from checkpoint: /var/folders/cv/xkhzqyq927df49h7pvkkv56m0000gn/T/checkpoint_tmp_yth394ro
(TrainableForPBT pid=67961) 2022-11-21 23:10:54,555	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 26, '_timesteps_total': None, '_time_total': 3.8244516849517822, '_episodes_total': None}


Result for TrainableForPBT_05d17_00013:
  date: 2022-11-21_23-10-54
  done: false
  experiment_id: c01173ebf45c4e309b7e8cf37b701cc6
  hostname: localhost
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 67961
  r2_score: 0.5014174660633255
  time_since_restore: 0.21303296089172363
  time_this_iter_s: 0.21303296089172363
  time_total_s: 4.037484645843506
  timestamp: 1669057854
  timesteps_since_restore: 0
  training_iteration: 27
  trial_id: 05d17_00013
  warmup_time: 0.005257844924926758
  
Result for TrainableForPBT_05d17_00012:
  date: 2022-11-21_23-10-54
  done: false
  experiment_id: 6106e214ad57445e98d3125a4a49402a
  hostname: localhost
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 67960
  r2_score: 0.8203316261338864
  time_since_restore: 0.6794319152832031
  time_this_iter_s: 0.6794319152832031
  time_total_s: 12.27336311340332
  timestamp: 1669057854
  timesteps_since_restore: 0
  training_iteration: 27
  trial_id: 05d17_00012
  warmup_time: 0.005780935287

(TrainableForPBT pid=67961) 2022-11-21 23:10:55,637	INFO trainable.py:668 -- Restored on 127.0.0.1 from checkpoint: /var/folders/cv/xkhzqyq927df49h7pvkkv56m0000gn/T/checkpoint_tmp_a3tj3mwj
(TrainableForPBT pid=67961) 2022-11-21 23:10:55,637	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 26, '_timesteps_total': None, '_time_total': 12.329398393630981, '_episodes_total': None}


Result for TrainableForPBT_05d17_00009:
  date: 2022-11-21_23-10-56
  done: true
  experiment_id: 6106e214ad57445e98d3125a4a49402a
  hostname: localhost
  iterations_since_restore: 4
  node_ip: 127.0.0.1
  pid: 67950
  r2_score: 0.8203316261338864
  time_since_restore: 2.7882325649261475
  time_this_iter_s: 0.7265317440032959
  time_total_s: 15.531867027282715
  timestamp: 1669057856
  timesteps_since_restore: 0
  training_iteration: 30
  trial_id: 05d17_00009
  warmup_time: 0.008919954299926758
  
Result for TrainableForPBT_05d17_00014:
  date: 2022-11-21_23-10-56
  done: false
  experiment_id: 6106e214ad57445e98d3125a4a49402a
  hostname: localhost
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 67961
  r2_score: 0.8203316261338864
  time_since_restore: 0.7028698921203613
  time_this_iter_s: 0.7028698921203613
  time_total_s: 13.032268285751343
  timestamp: 1669057856
  timesteps_since_restore: 0
  training_iteration: 27
  trial_id: 05d17_00014
  warmup_time: 0.0052578449249

2022-11-21 23:10:58,290	INFO tune.py:758 -- Total run time: 89.29 seconds (86.84 seconds for the tuning loop).


Now, we can interact with results using ResultGrid object (analysis). Using get_best_result we can get best result from the all trials. Also I will show you some useful results from ResultGrid.

In [16]:
best_trial_id = analysis._experiment_analysis.best_trial.trial_id
best_result = analysis.get_best_result()
best_result_score = best_result.metrics['r2_score']
best_config = best_result.config
best_checkpoint = best_result.checkpoint
best_checkpoint_dir = best_result.checkpoint.as_directory()
print(f"BEST TRIAL ID: {best_trial_id}")
print(f"BEST RESULT SCORE: {best_result_score}")
print(f"BEST CHECKPOINT DIRECTORY: {best_checkpoint}")

BEST TRIAL ID: 05d17_00001
BEST RESULT SCORE: 0.8203316261338864
BEST CHECKPOINT DIRECTORY: Checkpoint(local_path=/Users/admin/Desktop/Dressler/Publications/pbt_experiment/TrainableForPBT_05d17_00001_1_learning_rate=0.0005,max_depth=6,num_leaves=47_2022-11-21_23-09-36/checkpoint_000030)
